#### Ejercicio 7

Escribir un código para estimar la norma infinito de una matriz, usar la fórmula cerrada. Cómo
serı́a un código si no tenemos una fórmula cerrada? Comparar.

##### Solución

Partiendo de la definición de norma infinito para una matriz a partir de la norma vectorial:

$$
\left\| A \right\|_{\infty} = \sup \left\{ \left\| Ax \right\|_{\infty} : \left\| x \right\|_{\infty} = 1 \right\}
$$

Y aplicando la fórmula cerrada para una matriz 
cuadrada de dimensión $n$ como:

$$
\left\| A \right\|_{\infty} = \max_{1 \le i \le n} \sum_{j=i}^n \left| a_{ij} \right|
$$

Podemos implementarla con la siguiente función `inf_norm_mat()`:

In [1]:
def inf_norm_mat(matrix):
    return max(
        sum(abs(elem) for elem in row) for row in matrix
    )

La verificamos para la matriz ejemplo de las notas de 
clase:

$$
\left( \begin{matrix} 1 & 1 \\ 3 & 0 \end{matrix} \right)
$$

En nuestro caso:

In [2]:
inf_norm_mat([[1,1],[3,0]])

3

En el caso de que no fuera posible obtener
la fórmula cerrada de nuestra norma (como sería
para el caso de una norma $p$ en general)
deberíamos aplicar un procedimiento estadístico
generando la mayor cantidad posible de vectores $x$
que posean norma 1. Definimo así una segunda versión
de nuestra función aprovechando que la función 
`matmul` del paquete `numpy` se comporta como
producto escalar si ambos parámetros son vectores
de una dimensión:

In [3]:
from random import randint
from numpy import matmul


def normalize_vec(vector, norm):
    return [elem / norm(vector) for elem in vector]


def inf_norm_vec(vector):
    return max(abs(elem) for elem in vector)


def inf_norm_mat2(matrix, sample_size):
    return max(
        inf_norm_vec(
            matmul(
                matrix, normalize_vec(vector, inf_norm_vec)
            )
        )
        for vector in (
            [
                randint(-100, 100)
                for _ in range(len(matrix[0]))
            ]
            for _ in range(sample_size)
        )
    )

Para determinadas matrices se puede obtener un resultado
aceptable con pocas muestras. Reutilizando nuestra matriz
de ejemplo:

In [4]:
inf_norm_mat2([[1,1],[3,0]], 10)

3.0

Sin embargo, otras matrices nos obligan a una mayor 
cantidad de muestra para objener un resultado 
cercano al de la fórmula cerrada. Con ayuda de nuestra
función auxiliar `table` que se lista en el Anexo, 
construiremos una tabla comparativa para los 
resultados obtenidos con una matriz en el cálculo
de su norma infinita con diferentes tamaños de 
muestra aleatorias de vectores de norma infinita 1:

In [5]:
from auxiliares import tabla

tabla(
    [10 ** x for x in range(6)],
    [
        "Tamaño de muestra",
        "$\\begin{Vmatrix}A\end{Vmatrix}_{\infty}$",
    ],
    [
        repr,
        lambda x: inf_norm_mat2([[1, -2, 5], [1, 2, 4]], x),
    ],
)

|Tamaño de muestra|$\begin{Vmatrix}A\end{Vmatrix}_{\infty}$|
|:-:|:-:
|1|4.689655172413794|
|10|6.383720930232558|
|100|7.3508771929824555|
|1000|7.9523809523809526|
|10000|7.947368421052632|
|100000|8.0|


#### Ejercicio 11

Escribir un código para resolver el sistema matricial $Ax = b$ donde $A$ es una matriz que es una
permutación de una matríz triangular superior.

##### Solución

Asumiendo que la matríz $A$ es una permutación de una matriz triangular
superior, aplicamos el algoritmo de resolución de una tal matriz para 
un vector de valores independientes, siguiendo un ordenamiento dado 
por una matriz de permutaciones $P$. Es decir, que resolvemos la matriz
triangular superior $P \times A$ para el vector de valores independientes
$P \times b$. Así, la implementación de la función `solve_triu` resulta:

In [6]:
from itertools import takewhile
from functools import reduce


def solve_triu(matrix, values, perm):
    n = len(matrix)
    perm = [row.index(1) for row in perm]
    return reduce(
        lambda r, i: [
            (
                values[perm[i]]
                - matmul(
                    matrix[perm[i]][i + 1 : n],
                    r,
                )
            )
            / matrix[perm[i]][i]
        ]
        + r,
        reversed(range(n)),
        [],
    )

Verifiquemos la función con el siguiente sistema:

$$\left(\begin{matrix} 1 & 2 & 3 \\ 0 & 5 & 4 \\ 0 & 0 & 6 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 7 \\ 8 \\ 9 \end{matrix}\right)$$

En código:

In [7]:
A = [[1,2,3],
     [0,5,4],
     [0,0,6]]
b = [7,8,9]
P = [[1,0,0],
     [0,1,0],
     [0,0,1]]
x = solve_triu(A, b, P)
print(x)

[1.7000000000000002, 0.4, 1.5]


Si nuestra solución es correcta se debe verificar que $P \times A \times x = P \times b$. 
En nuestro ejemplo aprovechamos que la matriz $P$ es la identidad, por lo que la 
verificación de la igualdad en código es:

In [8]:
matmul(A, solve_triu(A, b, P))

array([7., 8., 9.])

#### Ejercicio 14

Usar el proceso de eliminación de Gauss Escalado para encontrar la descomposición $P \cdot A = L \cdot U$ (comparar con Python) y resolver en cada uno de los casos.

$$\left(\begin{matrix} -1 & 1 & -4 \\ 2 & 2 & 0 \\ 3 & 3 & 2 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 0 \\ 1 \\ \frac{1}{2} \end{matrix}\right)$$

$$\left(\begin{matrix} 1 & 6 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 3 \\ 1 \\ 1 \end{matrix}\right)$$

##### Solución

Primero definimos nuestra implementación de descomposición $LU$ 
mediante el método de Gauss Escalado para una matriz cuadrada, de manera que nuestra 
función `lu` devolverá una matriz de permutaciones $P$ y una matriz 
que corresponde a la suma matriz triangular superior $U$ permutada 
más una matriz triangular inferior $L$ menos la matriz identidad, 
también permutada. Es decir que para mejor aprovechamiento
de memoria resolveremos la descomposición _in situ_ y nuestro 
resultado equivaldrá a $P^{-1} \times (L - I + U)$. Para reconstruir
nuestras matrices no hace falta calcular la inversa de la permutación
ya que es ella misma. La implementación de la función es:

In [9]:
def lu(matrix):
    n = len(matrix)
    permut = list(range(n))
    maxim = [max(row, key=abs) for row in matrix]
    for i in range(n - 1):
        pos, pivot = max(
            enumerate(permut[i:]),
            key=lambda j: abs(matrix[j[1]][i] / maxim[j[1]]),
        )
        permut[i + pos] = permut[i]
        permut[i] = pivot
        for k in permut[i + 1 :]:
            matrix[k][i] /= matrix[pivot][i]
            for l in range(i + 1, n):
                matrix[k][l] -= (
                    matrix[k][i] * matrix[pivot][l]
                )
    return (
        [
            [1 if i == p else 0 for i in range(n)]
            for p in permut
        ],
        matrix
    )

De manera que si la intención no es resolver un
sistema sino presentar las matrices correspondientes,
podremos utilizar funciones de copias de matrices
del paquete `numpy`. En nuestro caso:

In [10]:
from numpy import triu, tril, identity


def split_lu(matrix):
    P, G1 = lu(matrix)
    G2 = matmul(P, G1)
    return P, tril(G2, -1) + identity(len(G2)), triu(G2)

Compararemos nuestros resultados con los de la implementación de
la descomposición que realiza el paquete `scipy` de Python:

In [11]:
from scipy.linalg import lu as lu2
P1, L1, U1 = split_lu([[-1,1,-4],[2,2,0],[3,3,2]])
P2, L2, U2 = lu2([[-1,1,-4],[2,2,0],[3,3,2]])
print(P1, L1, U1, sep="\n", end="\n\n")
print(P2, L2, U2, sep="\n")

[[0, 1, 0], [1, 0, 0], [0, 0, 1]]
[[ 1.   0.   0. ]
 [-0.5  1.   0. ]
 [ 1.5  0.   1. ]]
[[ 2.  2.  0.]
 [ 0.  2. -4.]
 [ 0.  0.  2.]]

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.66666667  0.          1.        ]]
[[ 3.          3.          2.        ]
 [ 0.          2.         -3.33333333]
 [ 0.          0.         -1.33333333]]


Como podemos observar, las descomposiciones
son diferentes, lo que es esperable ya que
también difieren los pivotes elegidos. Puesto
que si existe una descomposición $LU$ entonces
existen infinitas, debemos verificar que ambas 
cumplan con la igualdad $A = P^{-1}LU$, teniendo
en cuenta que la inversa de la matriz permutaciones
es la propia matriz de permutaciones:

In [12]:
print(matmul(P1, matmul(L1,U1)), end="\n\n")
print(matmul(P2, matmul(L2,U2)))

[[-1.  1. -4.]
 [ 2.  2.  0.]
 [ 3.  3.  2.]]

[[-1.  1. -4.]
 [ 2.  2.  0.]
 [ 3.  3.  2.]]


Para resolver nuestros sistemas sólo nos falta resolver
el caso de una matriz triangular inferior. Su algoritmo
es muy simular al de la matriz triangular superior con
la salvedad de que en este caso asumiremos que la 
diagonal es la identidad a menos que se indique lo
contrario, mecanismo habitual para aprovechar la 
descomposición _in situ_. Veamos su implementación:

In [13]:
def solve_tril(matrix, values, perm, identity=True):
    n = len(matrix)
    perm = [row.index(1) for row in perm]
    return reduce(
        lambda r, i: r + [
            (
                values[perm[i]]
                - matmul(
                    matrix[perm[i]][:i],
                    r,
                )
            )
            / (1 if identity else matrix[perm[i]][i])
        ],
        range(n),
        [],
    )

Verifiquemos su funcionamiento con un ejemplo
en espejo al utilizado al verificar el 
algoritmo para solucionar la matriz triangular superior:

$$\left(\begin{matrix} 6 & 0 & 0 \\ 4 & 5 & 0 \\ 3 & 2 & 1 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 9 \\ 8 \\ 7 \end{matrix}\right)$$

En código:

In [14]:
A = [[6,0,0],
     [4,5,0],
     [3,2,1]]
b = [9,8,7]
P = [[1,0,0],
     [0,1,0],
     [0,0,1]]
solve_tril(A, b, P, identity=False)

[1.5, 0.4, 1.7000000000000002]

Si nuestra solución es correcta se debe verificar que $P \times A \times x = P \times b$. 
Nuevamente, en nuestro ejemplo aprovechamos que la matriz $P$ es la identidad, por lo que la 
verificación de la igualdad en código es:

In [15]:
matmul(A, solve_tril(A, b, P, identity=False))

array([9., 8., 7.])

Ahora estamos en condiciones de resolver nuestro
primer sistema:

$$\left(\begin{matrix} -1 & 1 & -4 \\ 2 & 2 & 0 \\ 3 & 3 & 2 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 0 \\ 1 \\ \frac{1}{2} \end{matrix}\right)$$

En código:

In [16]:
P, L, U = split_lu([[-1,1,-4],[2,2,0],[3,3,2]])
I = [[1,0,0],[0,1,0],[0,0,1]]
x = solve_triu(U, solve_tril(L, matmul(P,[0, 1, 1/2]), I), I)
print(x)

[1.25, -0.75, -0.5]


Verifiquemos el resultado ya que $A \times x = b $:

In [17]:
matmul([[-1,1,-4],[2,2,0],[3,3,2]], x)

array([0. , 1. , 0.5])

En el caso que queramos trabajar con las matrices
_in situ_, será necesario permutar el resultado
intermedio en vez del vector de valores 
independientes, de modo que:

In [18]:
P, G = lu([[-1,1,-4],[2,2,0],[3,3,2]])
x = solve_triu(G, matmul(P, solve_tril(G, [0, 1, 1/2], P)), P)
print(x)

[1.25, -0.75, -0.5]


Usaremos este método para el siguiente sistema:

$$\left(\begin{matrix} 1 & 6 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \end{matrix}\right) \times \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 3 \\ 1 \\ 1 \end{matrix}\right)$$

En código:

In [19]:
P, G = lu([[1,6,0],[2,1,0],[0,2,1]])
x = solve_triu(G, matmul(P, solve_tril(G, [3, 1, 1], P)), P)
print(x)

[-0.46590909090909094, 0.6818181818181819, -0.36363636363636365]


Finalmente, verificamos el resultado:

In [20]:
matmul([[1,6,0],[2,1,0],[0,2,1]], x)

array([ 3.625, -0.25 ,  1.   ])